We first need to do some setup so we can query the Azure API

In [2]:
import os

from azure.identity import DefaultAzureCredential
from azure.mgmt.securityinsight import SecurityInsights
from dotenv import load_dotenv


load_dotenv()


SUBSCRIPTION_ID = os.environ["SUBSCRIPTION_ID"]
RESOURCE_GROUP_NAME = os.environ["RESOURCE_GROUP_NAME"]
WORKSPACE_NAME = os.environ["WORKSPACE_NAME"]

Use the Default Azure Credential (via something like `az login`). In this case we are excluding VSCode because of a bug w/ cached creds and are instead, using creds loaded as env vars from above

In [3]:
credential = DefaultAzureCredential(exclude_visual_studio_code_credential=True)

Ccreate a Security Insights Client which is what Microsoft calls the parent interface for Sentinel Rules. Note this is scoped to a subscription.

In [5]:
security_insights_client = SecurityInsights(credential, SUBSCRIPTION_ID)

Create an `AlertRulesTemplateClient` to get the alert rules. It's built from the above insights client

In [6]:
alert_rules_template_client = security_insights_client.alert_rule_templates

Get all of the Rule Templates. Note this is scoped to a tuple of `(resource group, workspace)`

In [10]:
rule_templates = alert_rules_template_client.list(RESOURCE_GROUP_NAME, WORKSPACE_NAME)

Grab the first template returned for examination

In [11]:
example_rule_template = list(rule_templates)[0]

print(f"Name: {example_rule_template.name}")
print(f"Description : {example_rule_template.description}")
print(f"Tactics: {example_rule_template.tactics}")
print(f"Techniques: {example_rule_template.techniques}")

Name: 3fbc20a4-04c4-464e-8fcb-6667f53e4987
Description : Identifies evidence of brute force activity against a Windows 365 Cloud PC by highlighting multiple authentication failures and by a successful authentication within a given time window.
Tactics: ['CredentialAccess']
Techniques: ['T1110']


Azure Sentinel Rules already included mapped ATT&CK data